Source: https://medium.com/trading-data-analysis/trading-algorithm-that-doesnt-work-37e747f4c6a6

https://alpaca.markets/docs/trading/getting_started/

In [1]:
import requests as rq
import alpaca_trade_api as tradeapi
import yfinance as yf
import pandas_ta as ta

In [2]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [3]:
# SETTINGS
TRADER_BOT_NAME = 'Medium trading bot'

TRADER_API_KEY = 'PKL8U1B2NM68MWD6TF56'
TRADER_API_SECRET = 'qnTt5X9YNYNxcLSKgrUVDNOGqJVxqtPFZwgmfbBQ'
TRADER_API_URL = 'https://paper-api.alpaca.markets'

TELEGRAM_URL = 'https://api.telegram.org'
TELEGRAM_BOT_ID = '6033820870:AAFQeEDNQ7XlmHSD5cFxbLqKCwUti87Xphw'
TELEGRAM_CHAT_ID = '-1001564842941'

SCREENER_INTERVAL = '5m'
SCREENER_PERIOD = '250m'
SCREENER_NASDAQ_COUNT = 100

TAKE_PROFIT_DELTA = 0.01
CASH_LIMIT = 29000

In [ ]:
def send_message(message):
    response = rq.post(f'{TELEGRAM_URL}/{TELEGRAM_BOT_ID}/sendMessage?chat_id={TELEGRAM_CHAT_ID}&parse_mode=Markdown&text={message}')
    return response

In [ ]:
trading_client = TradingClient(TRADER_API_KEY, TRADER_API_SECRET, paper=True)

In [16]:
account = trading_client.get_account()
positions = trading_client.get_all_positions()
market_order = trading_client.submit_order(market_order_data)
positions

[]

In [6]:
market_order_data = MarketOrderRequest(
                      symbol="BTC/USD",
                      qty=1,
                      side=OrderSide.BUY,
                      time_in_force=TimeInForce.GTC
                  )
market_order = trading_client.submit_order(market_order_data)

In [ ]:
MarketOrderReqa

In [ ]:
send_message("hi")

In [ ]:
# FIRST: set up CheckStock function which will be used in the next step during the screening process

#Let me explain how it works:
#We load tickers for first 500 stocks from NASDAQ and 
#check each of that stock at this particular moment for several conditions.

#For the market that is gonna go DOWN (market is overbought):
#Current stock price should be above the upper band of BBANDS;
#Current RSI value is above 70.

#For the market that is gonna go UP (market is oversold):
#Current stock price should be below the lower band of BBANDS;
#Current RSI value is below 30.

def CheckStock(stock):
    data = {}
    try:
        df = yf.download(stock, period = SCREENER_PERIOD, interval = SCREENER_INTERVAL)
        if (len(df) > 0):
            df['RSI'] = ta.rsi(df['Close'], timeperiod=14)
            #In our case, to identify this pattern we use Bollinger Bands with period = 20 and non-standard deviation equals to 2.3. 
            #This number is important, believe me. I got this setup for BBANDS based on my own experiments. 
            #They work much better than the standard deviation equaling 2. 
            #Actually these numbers are so important that most investment firms protect them in their public relations and non-disclosure agreements.
            bbands = ta.bbands(df['Close'], length = 20, std=2.3)
            df['L'] = bbands['BBL_20_2.3']
            df['M'] = bbands['BBM_20_2.3']
            df['U'] = bbands['BBU_20_2.3']
          
            previous2_bar = df[-3:].head(1)
            previous_bar = df[-2:].head(1)
            current_bar = df[-1:]
            
            if current_bar['RSI'].values[0] > 70 and \
                current_bar['Close'].values[0] > current_bar['U'].values[0]:
                    data = { 'direction': 'DOWN', 'stock' : stock, 'stop_loss': round(max(previous_bar['High'].values[0], previous2_bar['High'].values[0], previous_bar['U'].values[0]), 2), \
                            'take_profit': round(min(previous_bar['Low'].values[0], previous2_bar['Low'].values[0], previous_bar['M'].values[0]), 2) }
            elif current_bar['RSI'].values[0] < 30 and \
                current_bar['Close'].values[0] < current_bar['L'].values[0]:
                    data = { 'direction': 'UP', 'stock' : stock, 'stop_loss': round(min(previous_bar['Low'].values[0], previous2_bar['Low'].values[0], previous_bar['L'].values[0]), 2), \
                            'take_profit': round(max(previous_bar['High'].values[0], previous2_bar['High'].values[0], previous_bar['M'].values[0]), 2) }
    except:
        print("ERROR")

    return data

In [ ]:
#stock = "TSLA"
#df = yf.download(stock, period = SCREENER_PERIOD, interval = SCREENER_INTERVAL)
#df

In [ ]:
def ScreenStocks(trader_api):
    assets = trader_api.list_assets(status='active', asset_class='us_equity')
    assets = [x for x in assets if x.shortable == True and x.exchange == 'NASDAQ']
    stocks = [x.symbol for x in assets][:SCREENER_NASDAQ_COUNT]

    screened = []
    for st in stocks:
        # Here we are calling the CheckStock function from earlier for all stocks in the list
        _stock = CheckStock(st)
        if _stock != {}:
            screened.append(_stock)
    screened = [x for x in screened if abs(x['stop_loss'] - x['take_profit']) > min(x['stop_loss'], x['take_profit']) * TAKE_PROFIT_DELTA]
    # returns an already screened list of stocks, according to the set parameters
    print(screened)
    return screened


In [ ]:
def Predict(stock):
    predictions = [100.24, 155.33, 140.55]

  # Here we have to organize communication between our algorithm and LSTM Model \
  # to get predictions by ticker for the particular stock.
  # But this is the question to the infrastructure.
  # I am gonna consider it in the next article "Infrastructure itself".

    return predictions

In [ ]:
# Trade with STOP_LOSS and TAKE_PROFIT
def Trade(api, stock, operation, shares_to_trade, take_profit, stop_loss):
    api.submit_order(symbol = stock, qty = shares_to_trade, side = operation, type = 'market',
                    order_class = 'bracket', time_in_force = 'day', 
                    take_profit = {'limit_price': take_profit},
                    stop_loss = {'stop_price': stop_loss})
    message = f'\n\t*{stock}*, qty _{shares_to_trade}_ \n\t\twere {operation}'
    send_message(f'{TRADER_BOT_NAME}: we entered the market with:' + message)
    return True

In [ ]:
#api.submit_order(symbol = stock, qty = shares_to_trade, side = operation, type = 'market',
#                    order_class = 'bracket', time_in_force = 'day', 
#                    take_profit = {'limit_price': take_profit},
#                    stop_loss = {'stop_price': stop_loss})

In [ ]:
# MAIN script
def medium_trader_go(request):
    trader_api = api.REST(TRADER_API_KEY, TRADER_API_SECRET, TRADER_API_URL)
    account = trader_api.get_account()
    clock = trader_api.get_clock()

    if bool(account) == True:
        message = f'''{TRADER_BOT_NAME}: for *{account.account_number}*
        current capital is _{account.portfolio_value}$_ 
        and non marginable buying power is _{account.non_marginable_buying_power}$_'''
        send_message(message)

    if clock.is_open == True:
        if float(account.non_marginable_buying_power) < CASH_LIMIT:
            message = f"{TRADER_BOT_NAME}: there is no cash on the account or limit reached!"
            send_message(message)
        else:
          # Screen stocks
          screened = ScreenStocks(trader_api)
          # Check limit and trade
          if len(screened) > 0:
            CASH_FOR_TRADE_PER_SHARE = (float(account.non_marginable_buying_power) - CASH_LIMIT) / len(screened)
            for item in screened:
                predictions = Predict(item['stock'])
                STOCK = item['stock']
                OPERATION = 'buy' if item['direction'] == 'UP' else 'sell'
                STOP_LOSS = min([item['stop_loss']] + predictions) if item['direction'] == 'UP' else max([item['stop_loss']] + predictions)
                TAKE_PROFIT = max([item['take_profit']] + predictions) if item['direction'] == 'UP' else min([item['take_profit']] + predictions)
                SHARE_PRICE = round(min(STOP_LOSS, TAKE_PROFIT), 2)
                SHARES_TO_TRADE = int(CASH_FOR_TRADE_PER_SHARE / SHARE_PRICE)
                try:
                    if abs(STOP_LOSS - TAKE_PROFIT) > SHARE_PRICE * TAKE_PROFIT_DELTA and SHARES_TO_TRADE > 0:
                        Trade(api, STOCK, OPERATION, SHARES_TO_TRADE, TAKE_PROFIT, STOP_LOSS)
                        print(f'\n{STOCK}: {STOP_LOSS}, {TAKE_PROFIT}, {OPERATION}, {SHARES_TO_TRADE}')
                except:
                    pass

    portfolio = trader_api.list_positions()
    if bool(portfolio) == True:
        message = f'{TRADER_BOT_NAME}: we have {len(portfolio)} opened positions.'
        for i in portfolio:
            message = message + f'\n\t*{i.symbol}*: qty {i.qty} {i.side} for _{i.market_value}$_ \n\t\t\tcurrent price _{i.current_price}$_ \n\t\t\tprofit _{i.unrealized_pl}$_'
        send_message(message)
  
    if clock.is_open == False:
        message = f"{TRADER_BOT_NAME}: the market is *CLOSED*, let's try later on!"
        send_message(message)

    return f'{TRADER_BOT_NAME}: DONE!'

In [ ]:
#trader_api = api.REST(TRADER_API_KEY, TRADER_API_SECRET, TRADER_API_URL)

In [ ]:
#ScreenStocks(trader_api)

In [7]:
account

NameError: name 'account' is not defined